In [8]:
import os

# Function to check if minsearch.py exists and download it if not
def check_and_download_minsearch():
    file_name = "minsearch.py"
    if os.path.exists(file_name):
        print(f"{file_name} already exists.")
    else:
        try:
            # Download minsearch.py using wget
            print(f"Downloading {file_name}...")
            !wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
            print("\nDownload complete.")
        except Exception as e:
            print(f"Failed to download {file_name}: {e}")

# Execute the function
check_and_download_minsearch()

minsearch.py already exists.


In [9]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [10]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [11]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [12]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [13]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [14]:
llm('write that this is a test')

' This message serves as an automated or placeholder text to indicate where in your system the capability to generate human-like responses might be tested. In most applications, such texts are used by developers and quality assurance teams during testing phases before actual user content goes live for several reasons:\n\n1. **Validation of AI Capabilities** - To ensure that any natural language processing (NLP) or artificial intelligence model can understand contexts like this message to decide when it should generate responses without real data input. It\'s a way to validate the system in cases where actual user interaction cannot happen, for legal or ethical reasons.\n\n2. end-to-end testing - AI systems often need rigorous and comprehensive tests that simulate as many scenarios (like this message) throughout their entire workflow from start until finish without real data input involved to validate the system\'s behavior in every possible scenario which is beneficial for finding bugs

In [15]:
print(_)

 This message serves as an automated or placeholder text to indicate where in your system the capability to generate human-like responses might be tested. In most applications, such texts are used by developers and quality assurance teams during testing phases before actual user content goes live for several reasons:

1. **Validation of AI Capabilities** - To ensure that any natural language processing (NLP) or artificial intelligence model can understand contexts like this message to decide when it should generate responses without real data input. It's a way to validate the system in cases where actual user interaction cannot happen, for legal or ethical reasons.

2. end-to-end testing - AI systems often need rigorous and comprehensive tests that simulate as many scenarios (like this message) throughout their entire workflow from start until finish without real data input involved to validate the system's behavior in every possible scenario which is beneficial for finding bugs or une

In [16]:
llm('Building RAG course')

" To create a Responsibility Accounting and Governance (RAG) course, the curriculum should encompass theoretical knowledge as well as practical skills to enable students to understand financial control systems. Below is an outline for such a course:\n\n### Course Title: Building RAG - A Pathway to Financial Control & Responsibility Accounting Excellence\n\n#### Module 1: Introduction to Responsibility Accounting (3 weeks)\n- Understanding the basics of accounting and financial reporting.\n- Importance of responsibility accounting in management decision-making processes.\n- Case studies on traditional vs. RAG systems within businesses across various industries.\n\n#### Module 2: The Principles Behind Responsibility Accounting (3 weeks)\n- Detailed study of the principles that govern responsibility accounting, such as decentralization and performance evaluation.\n- Understanding different types of cost centers, profit centers, revenue centers, etc., through lectures and class discussions

Now working!!!